In [1]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings

warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *

# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

In [3]:
# 读取数据
tcc = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = [
    'gender',
    'SeniorCitizen',
    'Partner',
    'Dependents',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling',
    'PaymentMethod',
]

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges'] = (
    tcc['TotalCharges'].apply(lambda x: x if x != ' ' else np.nan).astype(float)
)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No', value=0, inplace=True)

In [4]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

In [5]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month'] - 1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month'] - 1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(
    enc.transform(X_train_seq).toarray(), columns=cate_colName(enc, seq_new, drop=None)
)

X_test_seq = pd.DataFrame(
    enc.transform(X_test_seq).toarray(), columns=cate_colName(enc, seq_new, drop=None)
)

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [6]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(
    ord_enc.transform(X_train[category_cols]), columns=category_cols
)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(
    ord_enc.transform(X_test[category_cols]), columns=category_cols
)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

In [15]:
# 模型创建
class logit_threshold(BaseEstimator, ClassifierMixin, TransformerMixin):
    def __init__(
        self,
        penalty='l2',
        C=1.0,
        max_iter=1e8,
        solver='lbfgs',
        l1_ratio=None,
        class_weight=None,
        thr=0.5,
    ) -> None:
        self.penalty = penalty
        self.C = C
        self.max_iter = max_iter
        self.solver = solver
        self.l1_ratio = l1_ratio
        self.thr = thr
        self.class_weight = class_weight

    def fit(self, x, y):
        clf = LogisticRegression(
            penalty=self.penalty,
            C=self.C,
            solver=self.solver,
            l1_ratio=self.l1_ratio,
            class_weight=self.class_weight,
            max_iter=self.max_iter,
            random_state=12,
        )
        clf.fit(x, y)
        self.coef_ = clf.coef_
        self.clf = clf
        return self

    def predict_proba(self, x):
        return self.clf.predict_proba(x)

    def predict(self, x):
        return (self.clf.predict_proba(x)[:, 1] >= self.thr) * 1

In [16]:
logistic_pre = ColumnTransformer(
    [
        ('cat', preprocessing.OneHotEncoder(drop='if_binary'), category_cols),
        ('num', 'passthrough', numeric_cols),
    ]
)

num_pre = [
    'passthrough',
    preprocessing.StandardScaler(),
    preprocessing.KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans'),
]

# 实例化逻辑回归评估器
logistic_model = logit_threshold(max_iter=int(1e8))

# 设置机器学习流
logistic_pipe = make_pipeline(logistic_pre, logistic_model)

# 设置超参数空间
cw_l = [None, 'balanced']
logistic_param = [
    {
        'columntransformer__num': num_pre,
        'logit_threshold__thr': np.arange(0.1, 1, 0.1).tolist(),
        'logit_threshold__penalty': ['l1'],
        'logit_threshold__C': np.arange(0.1, 1.1, 0.1).tolist(),
        'logit_threshold__solver': ['saga'],
        'logit_threshold__class_weight': cw_l,
    },
    {
        'columntransformer__num': num_pre,
        'logit_threshold__thr': np.arange(0.1, 1, 0.1).tolist(),
        'logit_threshold__penalty': ['l2'],
        'logit_threshold__C': np.arange(0.1, 1.1, 0.1).tolist(),
        'logit_threshold__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
        'logit_threshold__class_weight': cw_l,
    },
]

# 实例化网格搜索评估器
logistic_search = GridSearchCV(
    estimator=logistic_pipe, param_grid=logistic_param, scoring='accuracy', n_jobs=-1
)

In [17]:
logistic_search.fit(X_train_OE, y_train)

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(drop='if_binary'),
                                                                         ['gender',
                                                                          'SeniorCitizen',
                                                                          'Partner',
                                                                          'Dependents',
                                                                          'PhoneService',
                                                                          'MultipleLines',
                                                                          'InternetService',
                                                                          'OnlineSecurity',
                                                                          'OnlineBackup',
                                                                          'DeviceProtection',
                                                                          'TechSupport',
                                                                          'StreamingTV',
                                                                          'StreamingMovies',
                                                                          'Contract',
                                                                          'PaperlessBil...
                          'logit_threshold__C': [0.1, 0.2, 0.30000000000000004,
                                                 0.4, 0.5, 0.6,
                                                 0.7000000000000001, 0.8, 0.9,
                                                 1.0],
                          'logit_threshold__class_weight': [None, 'balanced'],
                          'logit_threshold__penalty': ['l2'],
                          'logit_threshold__solver': ['lbfgs', 'newton-cg',
                                                      'sag', 'saga'],
                          'logit_threshold__thr': [0.1, 0.2,
                                                   0.30000000000000004, 0.4,
                                                   0.5, 0.6, 0.7000000000000001,
                                                   0.8, 0.9]}],
             scoring='accuracy')

In [18]:
logistic_search.best_score_

0.8104888764656977

In [19]:
logistic_search.score(X_train_OE, y_train), logistic_search.score(X_test_OE, y_test)

(0.8123816736084817, 0.7836456558773425)

In [20]:
tree_model = DecisionTreeClassifier(
    ccp_alpha=0,
    max_depth=5,
    max_leaf_nodes=8,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=12,
).fit(X_train_OE, y_train)

In [21]:
tree_model.score(X_train_OE, y_train), tree_model.score(X_test_OE, y_test)

(0.7991291177584249, 0.768313458262351)

In [22]:
RF_0 = RandomForestClassifier(
    max_depth=10,
    max_features='sqrt',
    max_leaf_nodes=None,
    max_samples=2000,
    min_samples_leaf=2,
    min_samples_split=7,
    n_estimators=97,
    random_state=12,
).fit(X_train_OE, y_train)

In [23]:
RF_0.score(X_train_OE, y_train), RF_0.score(X_test_OE, y_test)

(0.8483528966300644, 0.7955706984667802)

In [24]:
from joblib import dump, load

In [25]:
dump(logistic_search, 'logistic_search.joblib') 
dump(tree_model, 'tree_model.joblib') 
dump(RF_0, 'RF_0.joblib') 

['RF_0.joblib']

In [26]:
logistic_search = load('logistic_search.joblib') 
tree_model = load('tree_model.joblib') 
RF_0 = load('RF_0.joblib') 

In [27]:
logistic_search

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(drop='if_binary'),
                                                                         ['gender',
                                                                          'SeniorCitizen',
                                                                          'Partner',
                                                                          'Dependents',
                                                                          'PhoneService',
                                                                          'MultipleLines',
                                                                          'InternetService',
                                                                          'OnlineSecurity',
                                                                          'OnlineBackup',
                                                                          'DeviceProtection',
                                                                          'TechSupport',
                                                                          'StreamingTV',
                                                                          'StreamingMovies',
                                                                          'Contract',
                                                                          'PaperlessBil...
                          'logit_threshold__C': [0.1, 0.2, 0.30000000000000004,
                                                 0.4, 0.5, 0.6,
                                                 0.7000000000000001, 0.8, 0.9,
                                                 1.0],
                          'logit_threshold__class_weight': [None, 'balanced'],
                          'logit_threshold__penalty': ['l2'],
                          'logit_threshold__solver': ['lbfgs', 'newton-cg',
                                                      'sag', 'saga'],
                          'logit_threshold__thr': [0.1, 0.2,
                                                   0.30000000000000004, 0.4,
                                                   0.5, 0.6, 0.7000000000000001,
                                                   0.8, 0.9]}],
             scoring='accuracy')